## ML using XGBoost
Installations to run:

$ pip install --user xgboost

---------------------
CPU only

$ conda install -c conda-forge py-xgboost-cpu

--Or--

Use NVIDIA GPU

$ conda install -c conda-forge py-xgboost-gpu

In [17]:
# Dependencies
import seaborn as sns

import pandas as pd

import numpy as np

from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt

import warnings


warnings.filterwarnings("ignore")

In [18]:
#Load in dataset
df = pd.read_csv("data/test_100.csv")

# Review the DataFrame
df.head()

,Unnamed: 0,Amount,Use Chip,Merchant State,Errors?,Is Fraud?,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,...,Has Chip,Cards Issued,International,Online,Age_at_transaction,income_to_debt,day_of_week,timestamp,time_of_day,distances
0,0,134.09,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,6,1.030861e+09,Morning,0.000000
1,1,38.48,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,6,1.030863e+09,Morning,33.540588
2,2,120.34,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,0,1.030948e+09,Morning,33.540588
3,3,128.95,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,0,1.030989e+09,Afternoon,33.540588
4,4,104.71,Swipe Transaction,CA,No Error,0,29278,59696,127613,787,...,1,2,0,0,36,0.467789,1,1.031034e+09,Morning,0.000000


In [19]:
df.shape

(1203635, 21)

In [20]:
df.describe()

,Unnamed: 0,Amount,Is Fraud?,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Has Chip,Cards Issued,International,Online,Age_at_transaction,income_to_debt,day_of_week,timestamp,distances
count,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06,1.203635e+06
mean,6.278133e+05,5.342784e+01,1.051814e-03,2.682409e+04,5.203543e+04,6.960732e+04,7.243838e+02,3.537857e+00,9.066287e-01,1.595465e+00,6.098194e-03,1.509419e-01,4.723789e+01,1.523181e+06,3.003065e+00,1.336919e+09,2.076576e+02
std,3.638027e+05,7.893761e+01,3.241463e-02,1.935230e+04,3.576085e+04,6.445562e+04,6.081448e+01,1.618776e+00,2.909522e-01,4.909323e-01,7.785250e-02,3.579924e-01,1.580150e+01,8.205016e+06,1.999756e+00,1.608702e+08,8.219707e+02
min,0.000000e+00,2.000000e-02,0.000000e+00,1.005900e+04,1.086900e+04,0.000000e+00,5.660000e+02,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.800000e+01,3.046351e-01,0.000000e+00,8.283496e+08,0.000000e+00
25%,3.127375e+05,1.281000e+01,0.000000e+00,1.789800e+04,3.560200e+04,2.184400e+04,6.850000e+02,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,3.500000e+01,5.123189e-01,1.000000e+00,1.224319e+09,0.000000e+00
50%,6.256890e+05,3.540000e+01,0.000000e+00,2.191600e+04,4.250900e+04,6.368900e+04,7.230000e+02,3.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,4.500000e+01,7.407534e-01,3.000000e+00,1.353999e+09,0.000000e+00
75%,9.444515e+05,7.017000e+01,0.000000e+00,2.674800e+04,5.453700e+04,9.545400e+04,7.700000e+02,5.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.700000e+01,1.913292e+00,5.000000e+00,1.471149e+09,1.665762e+02
max,1.258232e+06,6.820200e+03,1.000000e+00,1.631450e+05,2.499250e+05,4.375330e+05,8.500000e+02,8.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,9.100000e+01,2.167400e+08,6.000000e+00,1.582934e+09,8.148505e+03


In [21]:
df.describe(exclude=np.number)

,Use Chip,Merchant State,Errors?,time_of_day
count,1203635,1203635,1203635,1203635
unique,3,127,20,4
top,Swipe Transaction,Online,No Error,Morning
freq,728231,181679,1184197,481179


In [22]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = df.drop('Is Fraud?', axis=1), df[['Is Fraud?']]

In [23]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [24]:
X.dtypes

Unnamed: 0                        int64
Amount                          float64
Use Chip                       category
Merchant State                 category
Errors?                        category
Per Capita Income - Zipcode       int64
Yearly Income - Person            int64
Total Debt                        int64
FICO Score                        int64
Num Credit Cards                  int64
Has Chip                          int64
Cards Issued                      int64
International                     int64
Online                            int64
Age_at_transaction                int64
income_to_debt                  float64
day_of_week                       int64
timestamp                       float64
time_of_day                    category
distances                       float64
dtype: object

In [25]:
# Apply RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

In [26]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

In [27]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [28]:
# Define hyperparameters
params = {"objective": "binary:logistic", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

In [29]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)
# Convert probabilities to binary class predictions
predictions = (preds > 0.5).astype(int)

In [30]:
rmse = mean_squared_error(y_test, preds, squared=False)

print(f"RMSE of the base model: {rmse:.3f}")


RMSE of the base model: 0.213


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Evaluate with classification metrics
y_test_np = y_test.to_numpy()
print("Accuracy:", accuracy_score(y_test_np, predictions))
print("Precision:", precision_score(y_test_np, predictions))
print("Recall:", recall_score(y_test_np, predictions))
print("F1 Score:", f1_score(y_test_np, predictions))
print("Confusion Matrix:\n", confusion_matrix(y_test_np, predictions))
print("Classification Report:\n", classification_report(y_test_np, predictions))

Accuracy: 0.9447077409162717
Precision: 0.9384615384615385
Recall: 0.953125
F1 Score: 0.9457364341085271
Confusion Matrix:
 [[293  20]
 [ 15 305]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94       313
           1       0.94      0.95      0.95       320

    accuracy                           0.94       633
   macro avg       0.94      0.94      0.94       633
weighted avg       0.94      0.94      0.94       633

